<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_daily_storytelling_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [ ]:
%cd /content/achatbot


In [ ]:
!bash scripts/pypi_achatbot.sh dev


In [ ]:
!pip install -q "dist/achatbot-0.0.7.2-py3-none-any.whl[daily_room_audio_stream,silero_vad_analyzer,sense_voice_asr,deep_translator,openai,google_ai,together_ai,tts_edge,queue]"


run bot task woker with bot.json, e.g.: dummy_bot.json

- use daily/livekit room stream, u can click bot joined the room url, to start chat with bot with audio and camera stream,
  - [daily](https://www.daily.co/) need DAILY_API_KEY
- use openai/groq/together.ai api llm model need api key
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [groq](https://groq.com/) GROQ_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
- use image gen model:
  - [openai](https://openai.com/) OPENAI_API_KEY
  - [together.ai](https://www.together.ai/) TOGETHER_API_KEY
  - [huggingface api](https://huggingface.co/docs/api-inference/tasks/text-to-image) HF_API_KEY
  - [HFStableDiffusion](https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation) HF_TOKEN/HF_API_KEY




In [ ]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False


In [12]:
!cat /content/daily_storytelling_bot.json

{
  "chat_bot_name": "DailyStoryTellingBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "llm": "together",
    "gen_img": "together",
    "tts": "edge"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "llm": {
      "tag": "openai_llm_processor",
      "base_url": "https://api.together.xyz/v1",
      "model": "Qwen/Qwen2.5-72B-Instruct-Turbo",
      "language": "zh-CN",
      "messages": []
    },
    "img_gen": {
      "tag": "TogetherImageGenProcessor",
      "args": {
        "gen_rate_s": 10,
        "model": "black-forest-labs/FLUX.1-schnell-Fr

In [13]:
!DAILY_API_KEY=$DAILY_API_KEY TOGETHER_API_KEY=$TOGETHER_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_storytelling_bot.json

{
    "chat_bot_name": "DailyStoryTellingBot",
    "config": {
        "asr": {
            "args": {
                "language": "zn",
                "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
            },
            "tag": "sense_voice_asr"
        },
        "img_gen": {
            "args": {
                "gen_rate_s": 10,
                "model": "black-forest-labs/FLUX.1-schnell-Free"
            },
            "tag": "TogetherImageGenProcessor"
        },
        "llm": {
            "base_url": "https://api.together.xyz/v1",
            "language": "zh-CN",
            "messages": [],
            "model": "Qwen/Qwen2.5-72B-Instruct-Turbo",
            "tag": "openai_llm_processor"
        },
        "tts": {
            "args": {
                "gender": "Male",
                "language": "zh-CN",
                "voice_name": "zh-CN-YunjianNeural"
            },
            "tag": "tts_edge"
        },
        "vad": {
            "args": {
          

In [ ]:
from google.colab import userdata
REDIS_PASSWORD=userdata.get('REDIS_PASSWORD')


In [ ]:
!LOG_LEVEL=debug REDIS_PASSWORD=$REDIS_PASSWORD python -m src.cmd.bots.main -f /content/task_bot.json